# La mobilité des étudiants dans l'enseignement supérieur

Ce projet a été réalisé dans le cadre du cours de 2ème année de l'ENSAE "Python pour la data science" par Mathieu CAMBON, Charlotte HIECQUE et Elise MENSCH. 

## Introduction

### Contexte
La plateforme Parcoursup affecte chaque année des milliers d'étudiants à une formation. Les formations sont nombreuses, et même si certaines ont le même objectif, chacune possède une identité propre pouvant inciter un étudiant à postuler à une formation plutôt qu'une autre, engendrant ainsi une certaine mobilité des étudiants. Pourtant, cette mobilité peut être contrainte par de nombreux facteurs. De plus, elle est inégalement répartie sur le territoire, avec certaines villes qui attirent un grand nombre de personnes tandis que d'autres engendrent une fuite des étudiants, qui se dirigent vers d'autres académies dans le cadre de leurs études supérieures. Il apparaît donc intéressant d'étudier la mobilité des étudiants lors de leur entrée en étude supérieure. 

### Problématique 
Quels sont les facteurs qui encouragent une mobilité lors de l'entrée dans l'éducation supérieure ? 

De manière plus précise, nous étudierons l'effet des facteurs suivants : 
- Un substitut du niveau des élèves à travers la mention au bac : les meilleurs élèves sont-ils plus mobiles que les autres ?
- La sélectivité de la formation : les formations sélectives recrutent-elles leurs étudiants géographiquement plus loin ?
- La différence entre le privé et le public : le statut de l'établissement attire-t-il des élèves de plus loin ?
- Le genre : il y a-t-il des différences de mobilité entre des filières majoritairement féminines et majoritairement masculines ?

### Données
Pour effectuer cette analyse, nous avons principalement utilisé les données parcoursup en opendata sur data.gouv. A partir de cette base que nous avons nettoyé, nous avons créé de nouvelles données afin d'enrichir notre analyse. 

### Plan